# Physiological Model Software (PMS) with FHIR Integration

## Overview

The Physiological Model Software (PMS) is a Jupyter Notebook application that leverages the Pulse Physiology Engine to simulate physiological scenarios and integrates with a Fast Healthcare Interoperability Resources (FHIR) server to manage the observations recorded during simulations.

This application allows the user to simulates physiological data for a patient and sends vital signs specifically  heart rate and respiratory rate in to the FHIR server.

## Prerequisites

Before you begin, ensure you have the following installed:
- Docker
- Git (optional, for cloning repository)

## Getting Started

### Clone the Repository (Optional)

If the project is hosted on a version control system like GitHub, clone the repository using:

```sh
git clone <repository-url>
cd <repository-folder>

Build the Docker Image
To build the Docker image for the project, navigate to the directory containing the Dockerfile and run:


docker build -t ppe_fhir .
Run the Docker Container
To run the Docker container with Jupyter Notebook:


docker run -p 8888:8888 ppe_fhir jupyter notebook --ip="0.0.0.0" 
This command maps port 8888 on your local machine to port 8888 on the container and starts the Jupyter Notebook server.

Access the Jupyter Notebook
After running the container, open a web browser and navigate to:


http://127.0.0.1:8888/notebook
Enter the token provided in the terminal to log in to the Jupyter Notebook interface.

Application Usage
FHIR Server Configuration
The FHIR server is configured with the following URL:


FHIR_SERVER_URL = "http://host.docker.internal:8180/fhir"
Use http://host.docker.internal:8180/fhir instead of http://localhost:8180/fhir or http://127.0.0.1:8180/fhir
```
- But why http://host.docker.internal:8180/fhir ?
#### Problem: Network Boundaries
- Container Isolation: The FHIR server inside the Docker container has its own network environment. localhost or 127.0.0.1 for the notebook application refers to your host machine, not the container.

- Accessing the Container: Your notebook application, running on your host machine, needs a way to reach inside the Docker container's network to communicate with the FHIR server.

##### Solution: host.docker.internal to the Rescue

- The Bridge: host.docker.internal is a special hostname specifically for this purpose. Docker provides it as a way to address your host machine from within containers.

- How it Works: Docker dynamically maps host.docker.internal to the correct IP address of your host machine, essentially allowing your notebook app to talk to the FHIR server as if they were on the same network.

#### Why it Matters
- Using the correct URL ensures your notebook application can successfully:
 - Send requests to the FHIR server within the container
 - Retrieve FHIR data needed for its functionality

- This URL is specially designed to allow communication from within a Docker container to the host machine.

#### Simulating Scenarios
- Run the PMS notebook within the Jupyter interface.
- Enter the patient_id when prompted by the notebook.
- Use the interactive buttons provided to simulate different physiological scenarios such as airway obstruction and clearing.
- The simulation results will be logged and sent to the FHIR server in real-time.
#### Troubleshooting
- If you encounter any issues:
    - Ensure Docker is running on your system.
    - Verify that the FHIR server is up and accessible at the specified URL.
    - Check the Docker container logs for any error messages or indications of problems.

- After running the code and entering the real patient Id, expect the initial state values registered in the FHIR server. After hitting the button Obstruct air for 20 seconds, the Fhir server will get the last available values and same is for any actions taken.

### UserInterface Component
##### This component is for handling control widgets, and display outputs.

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
class UserInterface:
    def __init__(self, simulation_core):
        self.simulation_core = simulation_core
        self.output = widgets.Output()
        self._create_widgets()
    
    # Create the widgets for the user interface, the buttons for obstructing and clearing the airway
    # and the output widget for displaying the patient's initial state of the patient and state after each action
    def _create_widgets(self):
        self.button_obstruct_airway = widgets.Button(description="Obstruct Airway for 20 sec")
        self.button_clear_airway = widgets.Button(description="Clear Airway for 20 sec")
        self.button_obstruct_airway.on_click(self._obstruct_airway)
        self.button_clear_airway.on_click(self._clear_airway)
        display(self.button_obstruct_airway, self.button_clear_airway, self.output)

    # Obstructs the airway for 20 seconds and displays the patient's state after the obstruction
    def _obstruct_airway(self, b):
        with self.output:
            clear_output()
            self.simulation_core.obstruct_airway()

    # Clears the airway for 20 seconds and displays the patient's state after the obstruction
    def _clear_airway(self, b):
        with self.output:
            clear_output()
            self.simulation_core.clear_airway()


### Simulation Core Component
##### This component is for handling the simulation core logic and initiating the PulseEngine and DataRequestManager.

In [ ]:
from pulse.engine.PulseEngine import PulseEngine
from pulse.cdm.engine import SEDataRequest, SEDataRequestManager
from pulse.cdm.scalars import FrequencyUnit
from pulse.cdm.patient_actions import SEAirwayObstruction
from datetime import datetime

# SimulationCore class is responsible for initializing the PulseEngine, loading the initial state of the patient
class SimulationCore:
    def __init__(self, patient_id, data_management):
        self.patient_id = patient_id
        self.pulse = PulseEngine()
        self.data_management = data_management
        self.data_requests = [
            SEDataRequest.create_physiology_request("HeartRate", unit=FrequencyUnit.Per_min),
            SEDataRequest.create_physiology_request("RespirationRate", unit=FrequencyUnit.Per_min),
        ]
        self.data_req_mgr = SEDataRequestManager(self.data_requests)
        self.data_req_mgr.set_results_filename("./test_results/UseCase1.csv")
        if not self.pulse.serialize_from_file("./data/states/Soldier@0s.pbb", self.data_req_mgr):
            print("Unable to load initial state file")
        else:
            self.results = self.pulse.pull_data()
        self.data_management.save_results_in_csv(self.results)
        self.data_management.FHIR_data_mapper(self.results)

    # Advances the time by 1 second and prints the patient's state after each action taken
    def _advance_time_and_record(self, seconds, action=None):
        if action:
            self.pulse.process_action(action)
        for second in range(1, seconds + 1):
            self.pulse.advance_time_s(1)
            self.results = self.pulse.pull_data()
            print(self.results)
        self.data_management.save_results_in_csv(self.results)
        self.data_management.FHIR_data_mapper(self.results)
    
    # Obstructs the patient's airway for 20 seconds
    def obstruct_airway(self):
        airway_obstruction = SEAirwayObstruction()
        airway_obstruction.set_comment("Patient Airway is obstructed")
        airway_obstruction.get_severity().set_value(1)
        print("Obstructing patient airway...")
        self._advance_time_and_record(20, airway_obstruction)
    
    # Clears the patient's airway for 20 seconds
    def clear_airway(self):
        airway_clear = SEAirwayObstruction()
        airway_clear.set_comment("Patient Airway is cleared")
        airway_clear.get_severity().set_value(0)
        print("Clearing patient airway...")
        self._advance_time_and_record(20, airway_clear)


#### DataManagement Component
##### The component is for handling FHIR mapping and CSV file management.

In [13]:
import csv
import os
import logging
import requests
from datetime import datetime
import pytz

# DataManagement class is responsible for saving the patient's data to a CSV file and mapping the data to FHIR resources

class DataManagement:
    def __init__(self, patient_id, config_file):
        self.FHIR_SERVER_URL = config_file.get_FHIR_server_url()
        self.time_zone = config_file.get_time_zone()
        self.patient_id = patient_id
        self.csv_file_path = "./test_results/UseCase1.csv"
        os.makedirs(os.path.dirname(self.csv_file_path), exist_ok=True)

    # Maps the patient's data to FHIR resources and sends the observations to the FHIR server
    def FHIR_data_mapper(self, results):
        timezone = pytz.timezone(self.time_zone)
        timezone_now = datetime.now(timezone)
        observations = [
            {"loinc_code": "8867-4", "display": "Heart Rate", "value": float(results['HeartRate (1/min)'][0]), "unit": "beats/minute"},
            {"loinc_code": "9279-1", "display": "Respiratory Rate", "value": float(results['RespirationRate (1/min)'][0]), "unit": "breaths/minute"}
        ]

        for obs in observations:
            observation_data = {
                "resourceType": "Observation",
                "status": "final",
                "effectiveDateTime": timezone_now.isoformat(),
                "code": {
                    "coding": [{"system": "http://loinc.org", "code": obs['loinc_code'], "display": obs['display']}]
                },
                "subject": {"reference": f"Patient/{self.patient_id}"},
                "valueQuantity": {
                    "value": obs['value'],
                    "unit": obs['unit'],
                    "system": "http://unitsofmeasure.org"
                }
            }
            response = requests.post(f"{self.FHIR_SERVER_URL}/Observation", json=observation_data, headers={"Content-Type": "application/fhir+json"})
            if response.status_code == 201:
                print(f"{obs['display']} value: {obs['value']}")            
            else:
                logging.error(f"Failed to add observation {obs['display']}, status code: {response.status_code}, reason: {response.text}")
    
    # Saves the patient's data to a CSV file
    def save_results_in_csv(self, results):
        with open(self.csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(results.keys())
            writer.writerows(zip(*results.values()))


In [14]:
import configparser
class ConfigFile:
    def __init__(self):
        self.config = configparser.ConfigParser()
        self.config.read("config.ini")
    def get_FHIR_server_url(self):
        return self.config['FHIR']['server_url']
    def get_time_zone(self):
        return self.config['TIME_ZONE']['time_zone']



##### Main Script
###### Finally, integrate these components in the main script:



In [ ]:
patient_id = input("Enter the patient ID: ")
print(" Initial state of the patient:")
print("------------------------------:\n")
config_file = ConfigFile()
data_management = DataManagement(patient_id,config_file)
simulation_core = SimulationCore(patient_id, data_management)
ui = UserInterface(simulation_core)